Store pharmacophores in LMDB for fast access

In [1]:
import sys
import os
import torch
from glob import glob

os.chdir('..')

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tqdm import tqdm
from src.tacogfn.data.pharmacophore import PharmacoDB
from src.tacogfn.data.pocket import PocketDB

New PharmacoDB

In [3]:
pharmacophore_paths = glob('dataset/new_pharmacophores/*.pt')
keys = [path.split('/')[-1].split('.pt')[0] for path in pharmacophore_paths]

In [4]:
!rm -r misc/new_pharmacophores_db.lmdb

In [5]:
db = PharmacoDB('misc/new_pharmacophores_db.lmdb')
db.add_data(pharmacophore_paths, keys)

15221it [00:00, 22188.04it/s]


In [8]:
failed_keys = []

for key in tqdm(keys):
    try:
        dataset = db.get_data_from_id(key)
    except:
        failed_keys.append(key)

100%|██████████| 15221/15221 [00:58<00:00, 258.17it/s]


In [14]:
failed_keys

['1a31_A',
 '1byi_A',
 '1seu_A',
 '2gll_B',
 '2glm_A',
 '2rd0_A',
 '3khj_B',
 '3lsw_A',
 '3zg3_A',
 '4yi5_A',
 '5gwi_A',
 '5yil_B',
 '6bdi_A']

In [10]:
import lmdb

env = lmdb.open(db.db_path, create=False)
with env.begin(write=True) as txn:
    for key, value in tqdm(txn.cursor()):
        print(key, value)
        break

0it [00:00, ?it/s]

b'132l_A' b'\x80\x04\x95\x07\x00\x01\x00\x00\x00\x00\x00}\x94(\x8c\x05nodes\x94]\x94(}\x94(\x8c\x05index\x94K\x00\x8c\x10interaction_type\x94\x8c\x0bHydrophobic\x94\x8c\x10hotspot_position\x94G@4\xfa\x8d\x80\x00\x00\x00G@\x15\xf8h\xc0\x00\x00\x00G\xbf\xc7e\xc4\x00\x00\x00\x00\x87\x94\x8c\x05score\x94G?\xe1\xa0\nw\x1a\x00\xa7\x8c\x06center\x94G@8K#\xa0\x00\x00\x00G@\x174\xd8\x00\x00\x00\x00G\xc0\x06_\xb5 \x00\x00\x00\x87\x94\x8c\x06radius\x94G?\xf2\xa4G\x16\x1f\xbd\x80\x8c\x07feature\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\xc0\x85\x94h\x11\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00\x03\x00\x00p1\xd0\xbd{\x12h?\xf9@\x90@\xc0\r\x1a=*!a?C?\xfe=.\xf0K\xbac5X\xbd4\xb6\xbb?Mb\x8a<\xf78\x8b\xbcs\xce4>q\xacU;\x8b \xab?c\xf3\xeb>R1v\xbd\xf8\xb4\x1b?p\x91\xb7>F2\n=\x93\xb7^?\xbe\x16\xd8=\x13

PharmacoDB

In [3]:
pharmacophore_paths = glob('dataset/crossdock_pharmacophores/*.pkl')
keys = [path.split('/')[-1].split('_rec')[0] for path in pharmacophore_paths]

In [4]:
!rm -r misc/pharmacophores_db.lmdb

In [5]:
# Add pharmacophores to database
db = PharmacoDB('misc/pharmacophores_db.lmdb')
db.add_data(pharmacophore_paths, keys)

15116it [00:05, 2744.97it/s]


In [6]:
db._purge_none_data()

15116it [00:00, 494047.37it/s]

Removed 0 pharmacophores.


PocketDB

In [4]:
split_file = torch.load('dataset/split_by_name.pt')

In [5]:
pairs = split_file['train'] + split_file['test']
key_to_path = {}

In [6]:
for pair in pairs:
    pocket_path = os.path.join('dataset/crossdocked_pocket10/' , pair[0])
    key = pocket_path.split('/')[-1].split('_rec')[0] 
    key_to_path[key] = pocket_path

In [7]:
keys = list(key_to_path.keys())
pocket_paths = [key_to_path[key] for key in keys]

In [10]:
# Add pharmacophores to database
pocket_db = PocketDB('misc/pocket_db.lmdb')
pocket_db.add_data(pocket_paths , keys)

15307it [01:32, 165.64it/s]
